In [ ]:


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
import pickle
import warnings
warnings.filterwarnings('ignore')

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

print("Deep Learning LSTM Model - Task 2")

# 1. Generate sequence data for LSTM
X, y = make_classification(n_samples=2000, n_features=50, n_informative=30, 
                          n_redundant=10, n_clusters_per_class=2, random_state=42)

# Reshape for LSTM: (samples, timesteps, features)
X_seq = X.reshape(2000, 50, 1)
y = tf.keras.utils.to_categorical(y, 2)

# 2. Train-validation-test split
X_temp, X_test, y_temp, y_test = train_test_split(X_seq, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

# 3. Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, 1)).reshape(X_train.shape)
X_val_scaled = scaler.transform(X_val.reshape(-1, 1)).reshape(X_val.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, 1)).reshape(X_test.shape)

# 4. Build Deep Learning LSTM Model
model = Sequential([
    Bidirectional(LSTM(64, return_sequences=True), input_shape=(50, 1)),
    Dropout(0.3),
    Bidirectional(LSTM(32)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(2, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Model architecture ready")

# 5. Training with callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=25,
    batch_size=32,
    verbose=0,
    callbacks=[reduce_lr]
)

# 6. Visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Loss curves
axes[0,0].plot(history.history['loss'], 'b-', linewidth=2, label='Train Loss')
axes[0,0].plot(history.history['val_loss'], 'r-', linewidth=2, label='Val Loss')
axes[0,0].set_title('LSTM Training: Loss Curves')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Accuracy curves
axes[0,1].plot(history.history['accuracy'], 'b-', linewidth=2, label='Train Acc')
axes[0,1].plot(history.history['val_accuracy'], 'r-', linewidth=2, label='Val Acc')
axes[0,1].set_title('LSTM Training: Accuracy Curves')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Final metrics
train_acc = history.history['accuracy'][-1]
val_acc = history.history['val_accuracy'][-1]
axes[1,0].bar(['Train Acc', 'Val Acc'], [train_acc, val_acc], 
              color=['#4CAF50', '#2196F3'], alpha=0.8, edgecolor='black')
axes[1,0].set_title('Final Performance')
axes[1,0].set_ylabel('Accuracy')

# Learning rate (approximate)
lr_changes = [0.001] + [0.001 * (0.2)**i for i in range(1, 10)]
axes[1,1].plot(lr_changes[:15], 'g-', linewidth=2)
axes[1,1].set_title('Learning Rate Schedule')
axes[1,1].set_ylabel('Learning Rate')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('lstm_training_curves.png', dpi=300, bbox_inches='tight')
plt.close()

# 7. Test evaluation
y_pred_proba = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_proba, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Confusion Matrix
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_test_labels, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True,
            xticklabels=['Class 0', 'Class 1'],
            yticklabels=['Class 0', 'Class 1'],
            annot_kws={'size': 20})
plt.title('LSTM Deep Learning - Confusion Matrix')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.savefig('lstm_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.close()

# 8. Save model and scaler
model.save('lstm_deep_model.h5')
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)





Deep Learning LSTM Model - Task 2
Train: (1200, 50, 1), Val: (400, 50, 1), Test: (400, 50, 1)

Model architecture ready


13/13 [==============================] - 2s 9ms/step
Task-2 complete. Files generated:
- lstm_deep_model.h5
- lstm_training_curves.png
- lstm_confusion_matrix.png
- scaler.pkl
